In [1]:
import time
import pandas as pd
from selenium import webdriver

In [60]:
driver = webdriver.Chrome("chromedriver.exe")

In [87]:
driver.switch_to.window(driver.window_handles[1])

In [101]:
mail_list = [
    { "std_name" : "", "mail" : "" },
    { "std_name" : "", "mail" : "" }
]

In [ ]:
result_list = []
for mail in mail_list:
    try:
        driver.find_element_by_css_selector("#searchWordTxt").clear()
        driver.find_element_by_css_selector("#searchWordTxt").send_keys(mail["mail"])
    except:
        driver.find_element_by_css_selector("#searchWord").clear()
        driver.find_element_by_css_selector("#searchWord").send_keys(mail["mail"])        
    driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sArea > div.tab_wrap.clfix > div > fieldset > div > div.btnArea.sInput_box > ul > li > span > input[type=button]").click()
    time.sleep(2)
    try:
        result_list.append({ 
            "자녀이름" : mail["std_name"],
            "이름" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > div > table > tbody > tr:nth-child(2) > td:nth-child(3) > div > div").text.strip().split(" / ")[0],
            "부서" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > div > table > tbody > tr:nth-child(3) > td:nth-child(2) > div > div.tb_text_area > a").text.strip(),
            "전화번호" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > table > tbody > tr:nth-child(1) > td > span > div > div.tb_text_area").text.strip(),
            "휴대전화" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > table > tbody > tr:nth-child(2) > td:nth-child(2) > div > div.tb_text_area").text.strip(),
            "이메일" : driver.find_element_by_css_selector("#popWrap > div.popContents.popCorp > div > div.sResult > div.pInfoView > div > div > table > tbody > tr:nth-child(2) > td:nth-child(4) > div > div.tb_text_area > a").text.strip(),
        })
    except:
        pass
df = pd.DataFrame(result_list)
df.to_excel("result.xlsx", index=False)
df

In [43]:
import six
import copy
from pptx.util import Pt
from pptx import Presentation
from pptx.dml.color import RGBColor

df = pd.read_excel("result.xlsx")

In [44]:
def duplicate_slide(prs, org_slide):
    #add_slide() 메서드를 이용해 새로운 슬라이드 생성한 후 개체를 복사해 옴
    copied_slide = prs.slides.add_slide(org_slide.slide_layout)
    for shape in org_slide.shapes:
        org_elment = shape.element
        new_element = copy.deepcopy(org_elment)
        copied_slide.shapes._spTree.insert_element_before(new_element, "p:extLst")
    for _, value in six.iteritems(org_slide.part.rels):
        if "notesSlide" not in value.reltype:
            copied_slide.part.rels.add_relationship(
                value.reltype,
                value._target,
                value.rId
            )
    return copied_slide

In [55]:
prs = Presentation("문서수발 네임태그.pptx")
org_slide = prs.slides[0]
df_row_cnt = 0
for i in range(0, round(len(df)/8)): #0~5
    new_sld = duplicate_slide(prs, org_slide)
    for shape in new_sld.shapes:
        if df_row_cnt < len(df):
            for para in shape.text_frame.paragraphs:
                if "타이틀" in para.text:
                    para.text = "「주니어 인공지능 온라인 캠프」"
                    para.font.size = Pt(16)
                elif "받는분" in para.text:
                    para.text = "받는분 : " + df.iloc[df_row_cnt, 1] + "(" + df.iloc[df_row_cnt, 0] + ") / " + df.iloc[df_row_cnt, 3] + ", " + df.iloc[df_row_cnt, 4] + ", " + df.iloc[df_row_cnt, 5]
                    para.font.size = Pt(12)
                elif "부서명" in para.text:
                    para.text = "부서명 : " + df.iloc[df_row_cnt, 2]
                    para.font.size = Pt(12)
                para.font.bold = True
                para.font.name = "맑은 고딕"
                para.font.color.rgb = RGBColor(0, 0, 0)   
            df_row_cnt += 1
prs.save("result.pptx")